Task3. Vectorize evaluating the loss function (e.g., try to express as much as possible in terms of matrix
operations), to do more training iterations in a reasonable amount of time.

We have already improved the loss function in task1, and it significantly speeds up the training process. Here we compare the training time of the vectorized and original loss function.

In [6]:
import numpy as np
from scipy.io import loadmat


def standardizeCols(M, mu=None, sigma2=None):
    M = M.astype(float)  # Ensure M is float for precision
    nrows, ncols = M.shape
    if mu is None or sigma2 is None:
        mu = np.mean(M, axis=0)
        sigma2 = np.std(M, axis=0)
        sigma2[sigma2 < np.finfo(float).eps] = 1  # Avoid division by zero
    S = M - mu  # Subtract mean
    if ncols > 0:
        S = S / sigma2  # Divide by standard deviation
    return S, mu, sigma2


def form_weights(w, nVars, nHidden, nLabels):
    offset = 0
    inputWeights = w[offset:nVars * nHidden[0]].reshape(nVars, nHidden[0])
    offset += nVars * nHidden[0]
    hiddenWeights = []
    for h in range(1, len(nHidden)):
        size = nHidden[h-1] * nHidden[h]
        hiddenWeights.append(
            w[offset:offset+size].reshape(nHidden[h-1], nHidden[h]))
        offset += size
    outputWeights = w[offset:offset + nHidden[-1]
                      * nLabels].reshape(nHidden[-1], nLabels)
    return inputWeights, hiddenWeights, outputWeights


def sech2(x):
    return 1 - np.tanh(x)**2

In [7]:
# Original version
def MLPclassificationPredict(w, X, nHidden, nLabels):
    nInstances, nVars = X.shape

    # Form Weights
    inputWeights, hiddenWeights, outputWeights = form_weights(
        w, nVars, nHidden, nLabels)

    # Compute Output
    y = np.zeros((nInstances, nLabels))
    for i in range(nInstances):
        ip = [X[i] @ inputWeights]
        fp = [np.tanh(ip[0])]
        for h in range(1, len(nHidden)):
            ip.append(fp[h-1] @ hiddenWeights[h-1])
            fp.append(np.tanh(ip[h]))
        y[i] = fp[-1] @ outputWeights

    # Pick the class with the highest score
    y = np.argmax(y, axis=1, keepdims=True) + 1
    return y


def MLPclassificationLoss(w, X, y, nHidden, nLabels):
    # X, y are all 2-D arrays
    nInstances, nVars = X.shape
    inputWeights, hiddenWeights, outputWeights = form_weights(
        w, nVars, nHidden, nLabels)

    f = 0  # Loss
    gInput = np.zeros_like(inputWeights)  # Gradient for input weights
    # Gradient for hidden weights
    gHidden = [np.zeros_like(hw) for hw in hiddenWeights]
    gOutput = np.zeros_like(outputWeights)  # Gradient for output weights

    # Compute Output
    for i in range(nInstances):
        ip = [None] * len(nHidden)
        fp = [None] * len(nHidden)

        ip[0] = X[i, :] @ inputWeights  # 1-D array
        fp[0] = np.tanh(ip[0])  # 1-D array
        for h in range(1, len(nHidden)):
            ip[h] = fp[h-1] @ hiddenWeights[h-1]
            fp[h] = np.tanh(ip[h])
        yhat = fp[-1] @ outputWeights

        relativeErr = yhat - y[i, :]
        f += np.sum(relativeErr**2)

        err = 2 * relativeErr

        # Output Weights Gradient
        gOutput += np.outer(fp[-1], err)

        if len(nHidden) > 1:
            backprop = np.zeros_like(fp[-1])
            for c in range(nLabels):
                backprop[c] = err[c] * sech2(ip[-1]) * outputWeights[:, c].T
                gHidden[-1] += np.outer(fp[-2], backprop[c])
            backprop = np.sum(backprop, axis=0)  # Sum over classes

            # Other Hidden Layer Gradients
            for h in range(len(nHidden)-2, -1, -1):
                backprop = (backprop @ hiddenWeights[h].T) * sech2(ip[h])
                if h > 0:
                    gHidden[h-1] += np.outer(fp[h-1], backprop)
                else:
                    # Input weights
                    gInput += np.outer(X[i, :], backprop)
        else:
            # Input Weights Gradient for single hidden layer case
            for c in range(nLabels):
                gInput += err[c] * \
                    np.outer(X[i], sech2(ip[-1]) * outputWeights[:, c].T)

    # Put Gradient into vector
    g = np.zeros_like(w)
    g[:nVars * nHidden[0]] = gInput.reshape(-1, 1)
    offset = nVars * nHidden[0]
    for h in range(len(nHidden)-1):
        g[offset: offset + nHidden[h] * nHidden[h+1]
          ] = gHidden[h].reshape(-1, 1)
        offset += nHidden[h] * nHidden[h+1]
    g[offset: offset + nHidden[-1] * nLabels] = gOutput.reshape(-1, 1)

    return f, g

In [8]:
# Faster version
def MLPclassificationPredict_fast(w, X, nHidden, nLabels):
    nInstances, nVars = X.shape
    inputWeights, hiddenWeights, outputWeights = form_weights(w, nVars, nHidden, nLabels)
    activations = X
    for h in range(len(nHidden)):
        activations = np.tanh(activations @ (inputWeights if h == 0 else hiddenWeights[h-1]))
    y = activations @ outputWeights
    y = np.argmax(y, axis=1, keepdims=True) + 1
    return y


def MLPclassificationLoss_fast(w, X, y, nHidden, nLabels):
    nInstances, nVars = X.shape
    inputWeights, hiddenWeights, outputWeights = form_weights(w, nVars, nHidden, nLabels)

    activations = [X]
    for h in range(len(nHidden)):
        z = activations[-1] @ (inputWeights if h == 0 else hiddenWeights[h-1])
        a = np.tanh(z)
        activations.append(a)
    yhat = activations[-1] @ outputWeights
    f = np.sum((yhat - y)**2)  # Loss

    gOutput = 2 * activations[-1].T @ (yhat - y)
    gHidden = []
    delta = 2 * (yhat - y) @ outputWeights.T * sech2(activations[-1])
    for h in range(len(nHidden) - 1, 0, -1):
        gHidden.append(activations[h].T @ delta)
        delta = (delta @ hiddenWeights[h-1].T) * sech2(activations[h])
    gHidden.append(activations[0].T @ delta)
    gHidden.reverse()

    gradients = [gHidden[0].flatten()]
    for g in gHidden[1:]:
        gradients.append(g.flatten())
    gradients.append(gOutput.flatten())
    g = np.concatenate(gradients)

    return f, g.reshape(-1, 1)

In [9]:
data = loadmat('digits.mat')
X = data['X']
y = data['y'].flatten()
yvalid = data['yvalid']
ytest = data['ytest']

n, d = X.shape  # 5000, 256
nLabels = np.max(y)  # 10
yExpanded = 2 * np.eye(nLabels)[y - 1] - 1  # turn into one-hot vector
t = data['Xvalid'].shape[0]  # 5000
t2 = data['Xtest'].shape[0]  # 1000


# Standardize columns and add bias
X, mu, sigma = standardizeCols(X)
X = np.hstack([np.ones((n, 1)), X])

# Apply the same transformation to the validation/test data
Xvalid, _, _ = standardizeCols(data['Xvalid'], mu, sigma)
Xvalid = np.hstack([np.ones((t, 1)), Xvalid])

Xtest, _, _ = standardizeCols(data['Xtest'], mu, sigma)
Xtest = np.hstack([np.ones((t2, 1)), Xtest])


nHidden = [10]
d += 1

nParams = d * nHidden[0]
nParams += sum(nHidden[h-1] * nHidden[h] for h in range(1, len(nHidden)))
nParams += nHidden[-1] * nLabels
maxIter = 100000
initialStepSize = 1e-3
decayRate = 1e-5
momentum = 0.9

In [10]:
import time

time_slow = time.time()
w = np.random.randn(nParams, 1)
w_diff = np.zeros_like(w)
for iter in range(0, maxIter):
    stepSize = initialStepSize * (1 / (1 + decayRate * iter))
    if (iter + 1) % (maxIter // 10) == 0:
        yhat = MLPclassificationPredict(w, Xvalid, nHidden, nLabels)
        validation_error = np.mean(yhat != yvalid)
        print(f'Training iteration = {iter + 1}, validation error = {validation_error:.6f}')
    i = np.random.randint(n)
    f, g = MLPclassificationLoss(
        w, X[i:i+1], yExpanded[i:i+1], nHidden, nLabels)
    w_diff = momentum * w_diff - stepSize * g
    w += w_diff

# Evaluate test error
yhat = MLPclassificationPredict(w, Xtest, nHidden, nLabels)
test_error = np.mean(yhat != ytest)
print(f'Test error with final model = {test_error:.6f}')
time_slow = time.time() - time_slow

Training iteration = 10000, validation error = 0.553800
Training iteration = 20000, validation error = 0.408000
Training iteration = 30000, validation error = 0.415800
Training iteration = 40000, validation error = 0.330000
Training iteration = 50000, validation error = 0.421800
Training iteration = 60000, validation error = 0.306000
Training iteration = 70000, validation error = 0.309600
Training iteration = 80000, validation error = 0.301800
Training iteration = 90000, validation error = 0.287600
Training iteration = 100000, validation error = 0.259800
Test error with final model = 0.248000


In [11]:
time_fast = time.time()
w = np.random.randn(nParams, 1)
w_diff = np.zeros_like(w)
for iter in range(0, maxIter):
    stepSize = initialStepSize * (1 / (1 + decayRate * iter))
    if (iter + 1) % (maxIter // 10) == 0:
        yhat = MLPclassificationPredict_fast(w, Xvalid, nHidden, nLabels)
        validation_error = np.mean(yhat != yvalid)
        print(f'Training iteration = {iter + 1}, validation error = {validation_error:.6f}')
    i = np.random.randint(n)
    f, g = MLPclassificationLoss_fast(
        w, X[i:i+1], yExpanded[i:i+1], nHidden, nLabels)
    w_diff = momentum * w_diff - stepSize * g
    w += w_diff

# Evaluate test error
yhat = MLPclassificationPredict_fast(w, Xtest, nHidden, nLabels)
test_error = np.mean(yhat != ytest)
print(f'Test error with final model = {test_error:.6f}')
time_fast = time.time() - time_fast

print(f'Time taken for slow version: {time_slow:.2f}s')
print(f'Time taken for fast version: {time_fast:.2f}s')
print(f'Fast version is {time_slow / time_fast:.2f} times faster')

Training iteration = 10000, validation error = 0.266000
Training iteration = 20000, validation error = 0.213400
Training iteration = 30000, validation error = 0.219800
Training iteration = 40000, validation error = 0.205600
Training iteration = 50000, validation error = 0.242800
Training iteration = 60000, validation error = 0.208200
Training iteration = 70000, validation error = 0.213000
Training iteration = 80000, validation error = 0.230800
Training iteration = 90000, validation error = 0.201400
Training iteration = 100000, validation error = 0.213200
Test error with final model = 0.220000
Time taken for slow version: 74.60s
Time taken for fast version: 10.43s
Fast version is 7.15 times faster
